# A solvis.filter demo

In [1]:
import os
import pathlib
import json
from IPython.display import GeoJSON

import nzshm_model as nm
import solvis

# try our new filtering
from solvis.filter import FilterSubsectionIds
from solvis.filter import FilterRuptureIds
from solvis.inversion_solution.typing import SetOperationEnum

warning openquake module dependency not available, maybe you want to install
                with nzshm-model[openquake]


Running without `toshi` options


In [9]:
# prepare our NSHM composite solution
current_model = nm.get_model_version("NSHM_v1.0.4")
slt = current_model.source_logic_tree

# load the composite_solution and choose the Crustal FaultSystemSolution
csol = solvis.CompositeSolution.from_archive("NSHM_v1.0.4_CompositeSolution.zip", slt)
solution = csol._solutions['CRU']
DROP_ZERO_RATES=True

### Filter Ruptures by Fault Name, Magnitude and Rate

In [10]:
help(FilterRuptureIds)
# help(FilterRuptureIds.for_parent_fault_names)

Help on class FilterRuptureIds in module solvis.filter.rupture_id_filter:

class FilterRuptureIds(solvis.filter.chainable_set_base.ChainableSetBase)
 |  FilterRuptureIds(solution: solvis.inversion_solution.typing.InversionSolutionProtocol, drop_zero_rates: bool = True)
 |  
 |  A helper class to filter ruptures, returning the qualifying rupture_ids.
 |  
 |  Class `for_*` methods all return set-like objects, making  it possible to combine results using
 |  `set` operands e.g. `union`, `intersection`, `difference`.
 |  
 |  Class `for_* methods are 'chainable', so that multiple `for_` method calls can be linked
 |  together (see example below). The  default join operation is "intersection" so
 |  that each chained method call is 'refining' results from the prior method call(s).
 |  This behaviour can be overridden using the `join_prior` argument.
 |  
 |  Examples:
 |      ```py
 |      # ruptures on any of faults A, B, with magnitude and rupture rate limits
 |      rupture_ids = Filter

In [11]:
# filter ruptures by Fault Name, Magnitude and Rate
FAULTS = ['Masterton', 'Ohariu'] #, 'BooBoo']
rids = FilterRuptureIds(solution)\
    .for_parent_fault_names(FAULTS)\
    .for_magnitude(4.5, 7.9)\
    .for_rupture_rate(min_rate=5e-7, max_rate=1e-3)

print(f'{len(rids)} ruptures match the filter criteria')

80 ruptures match the filter criteria


In [12]:
# filter subsections by the ruptures we've selected
subsections = FilterSubsectionIds(solution).for_rupture_ids(rids)

surfaces = solution.fault_surfaces()
filtered = surfaces[surfaces.FaultID.isin(subsections)]

# print(filtered.columns)
filtered[['FaultID', 'FaultName', 'geometry', 'Target Slip Rate']].head()

,FaultID,FaultName,geometry,Target Slip Rate
127,127,"Awatere: Northeast 1, Subsection 0","POLYGON ((173.1947 -42.0989, 173.2427 -42.0842...",0.004095
128,128,"Awatere: Northeast 1, Subsection 1","POLYGON ((173.30463 -42.0482, 173.3438 -42.025...",0.004455
129,129,"Awatere: Northeast 1, Subsection 2","POLYGON ((173.40868 -41.98975, 173.51341 -41.9...",0.004815
130,130,"Awatere: Northeast 1, Subsection 3","POLYGON ((173.51341 -41.93205, 173.61796 -41.8...",0.005085
131,131,"Awatere: Northeast 1, Subsection 4","POLYGON ((173.61796 -41.87425, 173.7217 -41.81...",0.005895


### Filter Ruptures by Polygon, Magnitude and Rate

In [6]:
# help(FilterRuptureIds.for_polygons)

In [13]:
from nzshm_common.location.location import location_by_id
from solvis import circle_polygon

WLG = location_by_id('WLG')
MRO = location_by_id('MRO')
polygon0 = circle_polygon(1e5, WLG['latitude'], WLG['longitude'])  # 100km circle around WLG
polygon1 = circle_polygon(2e5, MRO['latitude'], MRO['longitude'])  # 200km circle around MRO

rids = FilterRuptureIds(solution, drop_zero_rates=DROP_ZERO_RATES)\
    .for_polygons([polygon1, polygon0], join_polygons='intersection')\
    .for_magnitude(min_mag=4.5, max_mag=8.5)\
    .for_rupture_rate(max_rate=1e-4, min_rate=1e-9)
print(f'{len(rids)} ruptures match the filter criteria')

902 ruptures match the filter criteria


In [8]:
# filter subsections by the ruptures we've selected
subsections = FilterSubsectionIds(solution).for_rupture_ids(rids)
surfaces = solution.fault_surfaces()
filtered = surfaces[surfaces.FaultID.isin(subsections)]

# print(filtered.columns)
filtered[['FaultID', 'FaultName', 'geometry', 'Target Slip Rate']].head()



,FaultID,FaultName,geometry,Target Slip Rate
5,5,"Akatarawa, Subsection 0","POLYGON ((175.0372 -41.1225, 175.0698 -41.1007...",0.001373
6,6,"Akatarawa, Subsection 1","POLYGON ((175.08915 -41.07156, 175.1132 -41.03...",0.001373
12,12,"Alexander, Subsection 0","POLYGON ((172.7301 -42.9266, 172.8193 -42.8949...",0.000518
13,13,"Alexander, Subsection 1","POLYGON ((172.85428 -42.89473, 172.9173 -42.89...",0.000810
14,14,"Alexander, Subsection 2","POLYGON ((172.98295 -42.87444, 173.01 -42.8662...",0.001463
